In [1]:
import argparse
import os
import random
import sys
import numpy as np
import torch
import torchaudio.functional as F
import config

from utils import download_dataset_if_needed

sys.path.insert(0, "models/VGGish")

from models.VGGish.vggish_multichannel import VGGishMulti

import pandas as pd
import numpy as np

import os

sys.path.insert(0, 'datasets/')

from datasets.dataset import Dataset
from datasets import livingroom
from datasets.roomsetup import RoomSetup
import matplotlib.pyplot as plt

import librosa

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

from utils import download_dataset_if_needed

In [2]:
download_dataset_if_needed()

path: ./LivingRoom_preprocessed_hack already exist, ignorign dataset downloading


In [3]:
DATASET_PATH = "LivingRoom_preprocessed_hack/Human1"

dr = Dataset(RoomSetup(livingroom.speaker_xyz,
                livingroom.mic_xyzs,
                livingroom.x_min,
                livingroom.x_max,
                livingroom.y_min,
                livingroom.y_max,
                livingroom.walls), DATASET_PATH)
livingroom.mic_xyzs

array([[-3343.275 , -3146.425 ,  1277.9375],
       [-3284.5375,  1587.5   ,  1277.9375],
       [  136.525 ,    53.975 ,  1277.9375],
       [  -76.2   , -3594.1   ,  1277.9375]])

In [4]:
centroid = np.load(os.path.join(DATASET_PATH, "centroid.npy"))
print("Shape of Centroid:")
print(centroid.shape)

#Loading Room Impulse Response (1000 human locations x 10 microphones x M time samples)
deconv = np.load(os.path.join(DATASET_PATH, "deconvoled_trim.npy"), mmap_mode='r')
print("Shape of RIRs:")
print(deconv.shape)

Shape of Centroid:
(1000, 2)
Shape of RIRs:
(1000, 4, 667200)


In [5]:
save_path = "models/VGGish/vggish_multi_model.pt"
error_path ="models/VGGish/vggish_multi_errors/"

# Choose if you want to use an already saved model
#vggish_multi = VGGishMulti(save_path=save_path, error_path=error_path)
vggish_multi = VGGishMulti(saved_model="models/VGGish/vggish_multi_model.pt", save_path=save_path, error_path=error_path)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(deconv, centroid, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [7]:
# Train the VGGishMulti model
# vggish_multi.fit(X_train, y_train, X_valid, y_valid, num_epochs=2)

In [ ]:
result = vggish_multi.predict(X_test, y_test)